# Similar2Logo groovy examples

This jupyter kernel presents the examples written in groovy distributed with Similar2Logo. Each example introduces a specific feature.

* [A first example with a passive turtle](#gpassive)

* [Adding a user-defined decision model to the turtles: The boids model](#gboids)

* [Dealing with marks: the turmite model](#gturmite)

* [Adding user-defined influence, reaction model and GUI: The segregation model](#gsegregation)

    To run the examples you must install the [scijava-jupyter-kernel](https://github.com/scijava/scijava-jupyter-kernel) and [compile Similar2Logo with maven](https://github.com/gildasmorvan/similar2logo/blob/master/README.md#compiling-similar2logo-with-maven).

Note that you have to restart the kernel before running a new example.

## <a name="gpassive"></a> A first example with a passive turtle

First we consider a simple example with a single passive agent.  

In [ ]:
#@repository("~/.m2/repository/")
#@dependency(group="fr.lgi2a", module="similar2logo-lib", version="0.9-SNAPSHOT")

import fr.lgi2a.similar.extendedkernel.simulationmodel.ISimulationParameters
import fr.lgi2a.similar.microkernel.AgentCategory
import fr.lgi2a.similar.microkernel.LevelIdentifier
import fr.lgi2a.similar.microkernel.SimulationTimeStamp
import fr.lgi2a.similar.microkernel.ISimulationModel.AgentInitializationData
import fr.lgi2a.similar.microkernel.agents.IAgent4Engine
import fr.lgi2a.similar.microkernel.levels.ILevel
import fr.lgi2a.similar2logo.kernel.initializations.LogoSimulationModel
import fr.lgi2a.similar2logo.kernel.model.Parameter
import fr.lgi2a.similar2logo.kernel.model.LogoSimulationParameters
import fr.lgi2a.similar2logo.kernel.model.agents.turtle.TurtleAgentCategory
import fr.lgi2a.similar2logo.kernel.model.agents.turtle.TurtleFactory
import fr.lgi2a.similar2logo.kernel.model.environment.LogoEnvPLS
import fr.lgi2a.similar2logo.lib.model.PassiveTurtleDecisionModel
import fr.lgi2a.similar2logo.lib.model.TurtlePerceptionModel
import fr.lgi2a.similar2logo.lib.probes.LogoRealTimeMatcher
import fr.lgi2a.similar2logo.lib.tools.html.Similar2LogoHtmlRunner

//Define the parameters of the simulation.
def simulationParameters = new LogoSimulationParameters() {
    
    @Parameter(
        name = "initial x", 
        description = "the initial position of the turtle on the x axis"
    )
    public double initialX = 10
    
    @Parameter(
        name = "initial y",
        description = "the initial position of the turtle on the y axis"
    )
    public double initialY = 10

    @Parameter(
        name = "initial speed",
        description = "the initial speed of the turtle"
    )
    public double initialSpeed = 0.1

    @Parameter(
        name = "initial acceleration",
        description = "the initial acceleration of the turtle"
    )
    public double initialAcceleration = 0

    @Parameter(
        name = "initial direction",
        description = "the initial direction of the turtle"
    )
    public double initialDirection = LogoEnvPLS.NORTH

}

//Define the simulation model of the simulation.
def simulationModel = new LogoSimulationModel(simulationParameters) {
    
    protected AgentInitializationData generateAgents(
        ISimulationParameters p,
        Map<LevelIdentifier, ILevel> levels
       ) {
           AgentInitializationData result = new AgentInitializationData()
           IAgent4Engine turtle = TurtleFactory.generate(
               new TurtlePerceptionModel(0, Double.MIN_VALUE, false, false, false),
               new PassiveTurtleDecisionModel(),
               new AgentCategory("passive", TurtleAgentCategory.CATEGORY),
               p.initialDirection,
               p.initialSpeed,
               p.initialAcceleration,
               p.initialX,
               p.initialY
           )
           result.agents.add turtle
           return result
       }
}

// Creation of the runner
def runner = new Similar2LogoHtmlRunner( )

// Configuration of the runner
runner.config.exportAgents = true

// Initialize the runner
runner.initializeRunner simulationModel

// Open the GUI
runner.showView( )

//Add a real time matcher probe
runner.addProbe "Real time matcher", new LogoRealTimeMatcher(20)

## <a name="gboids"></a> Adding a user-defined decision module to the turtles: The boids model

The [boids](https://en.wikipedia.org/wiki/Boids) (bird-oid) model has been invented by [Craig Reynolds](https://en.wikipedia.org/wiki/Craig_Reynolds_(computer_graphics)) in 1986 to simulate the flocking behavior of birds. It is based on 3 principles:
    
* separation: boids tend to avoid other boids that are too close,

* alignment: boids tend to align their velocity to boids that are not too close and not too far away,

* cohesion: bois tend to move towards boids that are too far away.

While these rules are essentially heuristic, they can be implemented defining three areas (repulsion, orientation, attraction) for each principle. 

* Boids change their orientation to get away from other boids in the repulsion area,

* Boids change their orientation and speed to match those of other boids in the orientation area,

* Boids change their orientation to get to other boids in the attraction area.

In [ ]:
#@repository("~/.m2/repository/")
#@dependency(group="fr.lgi2a", module="similar2logo-lib", version="0.9-SNAPSHOT")

import static java.lang.Math.*
import static fr.lgi2a.similar2logo.lib.tools.RandomValueFactory.strategy as rand
import fr.lgi2a.similar.extendedkernel.libs.abstractimpl.AbstractAgtDecisionModel
import fr.lgi2a.similar.extendedkernel.simulationmodel.ISimulationParameters
import fr.lgi2a.similar.microkernel.AgentCategory
import fr.lgi2a.similar.microkernel.LevelIdentifier
import fr.lgi2a.similar.microkernel.SimulationTimeStamp
import fr.lgi2a.similar.microkernel.ISimulationModel.AgentInitializationData
import fr.lgi2a.similar.microkernel.agents.IGlobalState
import fr.lgi2a.similar.microkernel.agents.ILocalStateOfAgent
import fr.lgi2a.similar.microkernel.agents.IPerceivedData
import fr.lgi2a.similar.microkernel.influences.InfluencesMap
import fr.lgi2a.similar.microkernel.levels.ILevel
import fr.lgi2a.similar2logo.kernel.initializations.LogoSimulationModel
import fr.lgi2a.similar2logo.kernel.model.LogoSimulationParameters
import fr.lgi2a.similar2logo.kernel.model.Parameter
import fr.lgi2a.similar2logo.kernel.model.agents.turtle.TurtleAgentCategory
import fr.lgi2a.similar2logo.kernel.model.agents.turtle.TurtleFactory
import fr.lgi2a.similar2logo.kernel.model.influences.ChangeDirection
import fr.lgi2a.similar2logo.kernel.model.influences.ChangeSpeed
import fr.lgi2a.similar2logo.kernel.model.levels.LogoSimulationLevelList
import fr.lgi2a.similar2logo.lib.model.TurtlePerceptionModel
import fr.lgi2a.similar2logo.lib.tools.html.Similar2LogoHtmlRunner

//Define the parameters of the simulation
def parameters = new LogoSimulationParameters() {
    
    @Parameter(name = "repulsion distance", description = "the repulsion distance")
    public double repulsionDistance = 6

    @Parameter(name = "attraction distance", description = "the attraction distance")
    public double attractionDistance = 14

    @Parameter(name = "orientation distance", description = "the orientation distance")
    public double orientationDistance = 10

    @Parameter(name = "maximal initial speed", description = "the maximal initial speed")
    public double maxInitialSpeed = 2

    @Parameter(name = "minimal initial speed", description = "the minimal initial speed")
    public double minInitialSpeed = 1

    @Parameter(name = "perception angle", description = "the perception angle in rad")
    public double perceptionAngle = PI

    @Parameter(name = "number of agents", description = "the number of boids in the simulation")
    public int nbOfAgents = 200

    @Parameter(name = "max angular speed", description = "the maximal angular speed in rad/step")
    public double maxAngle = PI/8
}

//Define the decision model of a boid
def decisionModel = new AbstractAgtDecisionModel(LogoSimulationLevelList.LOGO) {
    void decide(
        SimulationTimeStamp s, //the current simulation step
        SimulationTimeStamp ns, //the next simulation step
        IGlobalState gs, //the global state of the agent
        ILocalStateOfAgent pls, //the public local state of the boid
        ILocalStateOfAgent prls, //the private local state of the boid
        IPerceivedData pd, //the data perceived by the boid
        InfluencesMap i //the influences produced by the boid
    ) {
        if(!pd.turtles.empty) {
            def sc = 0, //the speed command
                sinoc = 0,//the sin of the orientation command
                cosoc = 0, //the cos of the orientation command
                n = 0 //the number of boids in the orientation area
            //computes the commands according to the area in which the perceived boid is located 
            pd.turtles.each{ boid ->
                switch(boid.distanceTo) {
                    //the repulsion area
                    case {it <= parameters.repulsionDistance}:
                        sinoc+=sin(pls.direction - boid.directionTo)
                        cosoc+=cos(pls.direction - boid.directionTo)
                        break
                    //the orientation area
                    case {it > parameters.repulsionDistance && it <= parameters.orientationDistance}:
                        sinoc+=sin(boid.content.direction - pls.direction)
                        cosoc+=cos(boid.content.direction - pls.direction)
                        sc+=boid.content.speed - pls.speed
                        n++
                        break
                    //the attraction area
                    case {it > parameters.orientationDistance && it <= parameters.attractionDistance}:
                        sinoc+=sin(boid.directionTo- pls.direction)
                        cosoc+=cos(boid.directionTo- pls.direction)
                        break
                }
            }
            //the orientation command
            def oc = atan2(sinoc/pd.turtles.size(), cosoc/pd.turtles.size())
            if (oc != 0) {
                //ceil the orientation command
                if(abs(oc) > parameters.maxAngle) oc = signum(oc)*parameters.maxAngle
                //emit a change direction influence
                i.add new ChangeDirection(s, ns, oc, pls)
            }
            //emit a change speed influence
            if (n > 0) i.add new ChangeSpeed(s, ns, sc/n, pls)
        }
    }
}

//Define the initial state of the simulation
def simulationModel = new LogoSimulationModel(parameters) {
    
    //Generate the agents
    protected AgentInitializationData generateAgents(
        ISimulationParameters p,
        Map<LevelIdentifier, ILevel> l
    ) {
        def result = new AgentInitializationData()
        //For each boid to be generated
        p.nbOfAgents.times {
            //generate the boid
            result.agents.add TurtleFactory.generate(
                new TurtlePerceptionModel(p.attractionDistance,p.perceptionAngle,true,false,false), //the perception model
                decisionModel, //the decision model
                new AgentCategory("b", TurtleAgentCategory.CATEGORY), //the category
                rand.randomAngle(), //the initial orientation
                p.minInitialSpeed + rand.randomDouble()*(p.maxInitialSpeed-p.minInitialSpeed),//the initial speed
                0,//the initial acceleration
                p.gridWidth/2,//the initial x position
                p.gridHeight/2 //the initial y position
            )
        }
        return result
    }
}

// Creation of the runner
def runner = new Similar2LogoHtmlRunner( )

// Configuration of the runner
runner.config.exportAgents = true

// Initialize the runner
runner.initializeRunner simulationModel

// Open the GUI
runner.showView( )

## <a name="gturmite"></a> Dealing with marks: the turmite model

The [turmite model](https://en.wikipedia.org/wiki/Langton's_ant), developed by [Christopher Langton](https://en.wikipedia.org/wiki/Christopher_Langton) in 1986, is a very simple mono-agent model that exhibits an emergent behavior. It is based on 2 rules:

* If the turmite is on a patch that does not contain a mark, it turns right, drops a mark, and moves forward,

* If the turmite is on a patch that contains a mark, it turns left, removes the mark, and moves forward.

In [ ]:
#@repository("~/.m2/repository/")
#@dependency(group="fr.lgi2a", module="similar2logo-lib", version="0.9-SNAPSHOT")

import static java.lang.Math.*
import java.awt.geom.Point2D

import fr.lgi2a.similar.extendedkernel.libs.abstractimpl.AbstractAgtDecisionModel
import fr.lgi2a.similar.extendedkernel.simulationmodel.ISimulationParameters
import fr.lgi2a.similar.microkernel.AgentCategory
import fr.lgi2a.similar.microkernel.LevelIdentifier
import fr.lgi2a.similar.microkernel.SimulationTimeStamp
import fr.lgi2a.similar.microkernel.ISimulationModel.AgentInitializationData
import fr.lgi2a.similar.microkernel.agents.IAgent4Engine
import fr.lgi2a.similar.microkernel.agents.IGlobalState
import fr.lgi2a.similar.microkernel.agents.ILocalStateOfAgent
import fr.lgi2a.similar.microkernel.agents.IPerceivedData
import fr.lgi2a.similar.microkernel.influences.InfluencesMap
import fr.lgi2a.similar.microkernel.levels.ILevel
import fr.lgi2a.similar2logo.kernel.initializations.LogoSimulationModel
import fr.lgi2a.similar2logo.kernel.model.LogoSimulationParameters
import fr.lgi2a.similar2logo.kernel.model.agents.turtle.TurtleAgentCategory
import fr.lgi2a.similar2logo.kernel.model.agents.turtle.TurtleFactory
import fr.lgi2a.similar2logo.kernel.model.environment.LogoEnvPLS
import fr.lgi2a.similar2logo.kernel.model.environment.Mark
import fr.lgi2a.similar2logo.kernel.model.influences.ChangeDirection
import fr.lgi2a.similar2logo.kernel.model.influences.DropMark
import fr.lgi2a.similar2logo.kernel.model.influences.RemoveMark
import fr.lgi2a.similar2logo.kernel.model.levels.LogoSimulationLevelList
import fr.lgi2a.similar2logo.lib.model.TurtlePerceptionModel
import fr.lgi2a.similar2logo.lib.probes.LogoRealTimeMatcher
import fr.lgi2a.similar2logo.lib.tools.html.Similar2LogoHtmlRunner

//Define the parameters of the simulation
def parameters = new LogoSimulationParameters(
    finalTime: new SimulationTimeStamp(100000)
)

//Define the decision model of the agents
def decisionModel = new AbstractAgtDecisionModel(LogoSimulationLevelList.LOGO) {
    void decide(
        SimulationTimeStamp s, //the current simulation step
        SimulationTimeStamp ns, //the next simulation step
        IGlobalState gs, //the global state of the agent
        ILocalStateOfAgent pls, //the public local state of the agent
        ILocalStateOfAgent prls, //the private local state of the agent
        IPerceivedData pd, //the data perceived by the agent
        InfluencesMap i //the influences produced by the agent
    ) {
        if(pd.marks.empty) i.with { //if the agent perceives no mark
            add new ChangeDirection(s, ns, PI/2, pls) //it turns pi/2 rad
            add new DropMark(s, ns, new Mark((Point2D) pls.location.clone(), null)) //and drops a mark
        } else i.with { //if the agent perceives a mark
            add new ChangeDirection(s, ns, -PI/2, pls) //it turns -pi/2 rad
            add new RemoveMark(s, ns,pd.marks.iterator().next().content) //and removes the mark
        }
    }
}

//Define the initial state of the simulation
def simulationModel = new LogoSimulationModel(parameters) {
    
    //Generate the agents
    protected AgentInitializationData generateAgents(
        ISimulationParameters simulationParameters, //the parameters of the simulation
        Map<LevelIdentifier, ILevel> levels //the levels of the simulation
    ) {
        //Create a new turmite agent
        def turmite = TurtleFactory.generate(
            new TurtlePerceptionModel(0, Double.MIN_VALUE, false, true, false), //a perception model that allows to perceive marks
            decisionModel, //the turmite decision model
            new AgentCategory("turmite", TurtleAgentCategory.CATEGORY), //the turmite category
            LogoEnvPLS.NORTH, //heading north
            1, //a speed of 1
            0, //an acceleration of 0
            10.5, 10.5 //located at 10.5, 10.5
        ),
        result = new AgentInitializationData() //create the agent initialization data
        result.agents.add turmite //add the turmite agent
        return result
    }
}

// Creation of the runner
def runner = new Similar2LogoHtmlRunner( )

// Configuration of the runner
runner.config.exportAgents = true
runner.config.exportMarks = true

// Initialize the runner
runner.initializeRunner simulationModel

//Add a real time matcher probe
runner.addProbe "Real time matcher", new LogoRealTimeMatcher(20)

// Open the GUI
runner.showView( )

## <a name="gsegregation"></a> Adding user-defined influence, reaction model and GUI: The segregation model

The segregation model has been proposed by [Thomas Schelling](https://en.wikipedia.org/wiki/Thomas_Schelling) in 1971 in his famous paper [Dynamic Models of Segregation](https://www.stat.berkeley.edu/~aldous/157/Papers/Schelling_Seg_Models.pdf). The goal of this model is to show that segregation can occur even if it is not wanted by the agents.

In our implementation of this model, turtles are located in the grid and at each step, compute an happiness index based on the similarity of other agents in their neighborhood. If this index is below a value, called here similarity rate, the turtle wants to move to an other location.

In [ ]:
#@repository("~/.m2/repository/")
#@dependency(group="fr.lgi2a", module="similar2logo-lib", version="0.9-SNAPSHOT")

import static java.lang.Math.*
import static fr.lgi2a.similar2logo.kernel.model.levels.LogoSimulationLevelList.LOGO
import static fr.lgi2a.similar2logo.lib.tools.RandomValueFactory.strategy as rand

import java.awt.geom.Point2D

import fr.lgi2a.similar.extendedkernel.levels.ExtendedLevel
import fr.lgi2a.similar.extendedkernel.libs.abstractimpl.AbstractAgtDecisionModel
import fr.lgi2a.similar.extendedkernel.libs.timemodel.PeriodicTimeModel
import fr.lgi2a.similar.extendedkernel.simulationmodel.ISimulationParameters
import fr.lgi2a.similar.microkernel.AgentCategory
import fr.lgi2a.similar.microkernel.LevelIdentifier
import fr.lgi2a.similar.microkernel.SimulationTimeStamp
import fr.lgi2a.similar.microkernel.ISimulationModel.AgentInitializationData
import fr.lgi2a.similar.microkernel.agents.IGlobalState
import fr.lgi2a.similar.microkernel.agents.ILocalStateOfAgent
import fr.lgi2a.similar.microkernel.agents.IPerceivedData
import fr.lgi2a.similar.microkernel.dynamicstate.ConsistentPublicLocalDynamicState
import fr.lgi2a.similar.microkernel.influences.IInfluence
import fr.lgi2a.similar.microkernel.influences.InfluencesMap
import fr.lgi2a.similar.microkernel.influences.RegularInfluence
import fr.lgi2a.similar.microkernel.levels.ILevel
import fr.lgi2a.similar2logo.kernel.initializations.LogoSimulationModel
import fr.lgi2a.similar2logo.kernel.model.LogoSimulationParameters
import fr.lgi2a.similar2logo.kernel.model.Parameter
import fr.lgi2a.similar2logo.kernel.model.agents.turtle.TurtleAgentCategory
import fr.lgi2a.similar2logo.kernel.model.agents.turtle.TurtleFactory
import fr.lgi2a.similar2logo.kernel.model.agents.turtle.TurtlePLSInLogo
import fr.lgi2a.similar2logo.kernel.model.environment.LogoEnvPLS
import fr.lgi2a.similar2logo.kernel.model.levels.LogoDefaultReactionModel
import fr.lgi2a.similar2logo.lib.model.TurtlePerceptionModel
import fr.lgi2a.similar2logo.lib.tools.html.Similar2LogoHtmlRunner

//Define the parameters of the simulation
def parameters = new LogoSimulationParameters() {
    @Parameter(
        name = "similarity rate",
        description = "the rate of same-color turtles that each turtle wants among its neighbors"
    )
    public double similarityRate = 3.0/8

    @Parameter(name = "vacancy rate", description = "the rate of vacant settling places")
    public double vacancyRate = 0.05

    @Parameter(name = "perception distance", description = "the perception distance of agents")
    public double perceptionDistance = sqrt(2)
}

//Define the specific influence of this model
class Move extends RegularInfluence {
    
    //The turtle's public local state that is going to change
    def target 
    
    //the category of the influence, used as a unique identifier in the reaction to determine the nature of the influence
    static final def CATEGORY = "move"
        
    Move(SimulationTimeStamp s, SimulationTimeStamp ns, TurtlePLSInLogo target) {
        super(CATEGORY, LOGO, s, ns)
        this.target = target
    }
}

//Define the decision model of an agent
def decisionModel = new AbstractAgtDecisionModel(LOGO) {
    void decide(
        SimulationTimeStamp s, //the current simulation step
        SimulationTimeStamp ns, //the next simulation step
        IGlobalState gs, //the global state of the agent
        ILocalStateOfAgent pls, //the public local state of the agent
        ILocalStateOfAgent prls, //the private local state of the agent
        IPerceivedData pd, //the data perceived by the agent
        InfluencesMap i //the influences produced by the agent
    ) {
        //compute the similarity rate
        def sr = 0
        pd.turtles.each{ agent -> if(agent.content.categoryOfAgent.isA(pls.categoryOfAgent)) sr++ } 
        if(!pd.turtles.empty) sr/= pd.turtles.size()
        //if the similarity rate is too low, the agent wants to move
        if(sr < parameters.similarityRate) i.add new Move(s, ns, pls)
    }
}

//Define the reaction model
def reactionModel = new LogoDefaultReactionModel() {
    //redefine the reaction function for regular influences
    public void makeRegularReaction(
        SimulationTimeStamp s, //the current simulation step
        SimulationTimeStamp ns, //the next simulation step
        ConsistentPublicLocalDynamicState cs, //the dynamic state of the simulation
        Set<IInfluence> influences, //the influences to process 
        InfluencesMap remainingInfluences //the influences that will remain i the dynamic state
    ) {
        def e = cs.publicLocalStateOfEnvironment, //the environment
            li = [], //the list of influences
            vacant = [] //the list of vacant housings	
        li.addAll influences //create the list of influences
        Collections.shuffle li //shuffle the list of influences
        for(x in 0..<e.width) for(y in 0..<e.height)
            if(e.getTurtlesAt(x, y).empty) vacant.add new Point2D.Double(x,y) //identify vacant housings
        Collections.shuffle vacant //shuffle the list of vacant housings
        def n = 0 
        li.any{ i -> //move lucky unhappy agents to vacant housings
            if(i.category == Move.CATEGORY) {
                e.turtlesInPatches[(int) i.target.location.x][(int) i.target.location.y].remove i.target
                e.turtlesInPatches[(int) vacant[n].x][(int) vacant[n].y].add i.target
                i.target.setLocation(vacant[n])
                if(++n >= vacant.size()) return true //stop when no more housing is available
            }
        }
    }
}

//Define the simulation model
def simulationModel = new LogoSimulationModel(parameters) {
    
    List<ILevel> generateLevels(ISimulationParameters p) {
        def logo = new ExtendedLevel(
            p.initialTime,
            LOGO,
            new PeriodicTimeModel(1,0, p.initialTime),
            reactionModel
        )
        def levelList = []
        levelList.add logo
        return levelList
    }

    AgentInitializationData generateAgents(ISimulationParameters p, Map<LevelIdentifier, ILevel> l) {
        def result = new AgentInitializationData()
        for(x in 0..<p.gridWidth) for(y in 0..<p.gridHeight)
            if(rand.randomDouble() >= p.vacancyRate) result.agents.add TurtleFactory.generate(
                new TurtlePerceptionModel(p.perceptionDistance, 2*PI, true, false, false),
                decisionModel,
                new AgentCategory(rand.randomBoolean() ? "a" :"b", TurtleAgentCategory.CATEGORY),
                0, 0, 0, x,y
            )
        return result
    }
}

//Define the custom GUI
def segregationgui = '''
    <canvas id='grid_canvas' class='center-block' width='400' height='400'></canvas>
    <script type='text/javascript'>
        drawCanvas = function (data) {
            var json = JSON.parse(data),
                canvas = document.getElementById('grid_canvas'),
                context = canvas.getContext('2d');
            context.clearRect(0, 0, canvas.width, canvas.height);
            for (var i = 0; i < json.agents.length; i++) {
                var centerX = json.agents[i].x * canvas.width;
                var centerY = json.agents[i].y * canvas.height;
                var radius = 2;
                if (json.agents[i].t == 'a') {
                    context.fillStyle = 'red';
                } else {
                    context.fillStyle = 'blue';
                }
                context.beginPath();
                context.arc(centerX, centerY, radius, 0, 2 * Math.PI, false);
                context.fill();
            }
        }
    </script>'''

//Creation of the runner
def runner = new Similar2LogoHtmlRunner( )

//Configuration of the runner
runner.config.exportAgents = true

// Initialize the GUI
runner.config.setCustomHtmlBodyFromString segregationgui

// Initialize the runner
runner.initializeRunner simulationModel

// Open the GUI
runner.showView( )